In [1]:
import os

cwd = os.getcwd()

cwd

'/Volumes/GoogleDrive/My Drive/Fordham/machine_learning/Assignment_5'

In [2]:
import pandas as pd
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import LocallyConnected1D,Reshape
from keras import backend as K


n_classes = 10
epochs = 12

train = pd.read_table("zip.train",delim_whitespace=True,header=None)

x_train, y_train = train.loc[:,1:257], train.loc[:,0]

test = pd.read_table("zip.test",delim_whitespace=True,header=None)

x_test, y_test = test.loc[:,1:257], test.loc[:,0]



print("Train: Rows: %d, Columns: %d" %(x_train.shape[0], x_train.shape[1]))
print("Test: Rows: %d, Columns: %d" %(x_test.shape[0], x_test.shape[1]))



/Users/yizhizhang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train: Rows: 7291, Columns: 256
Test: Rows: 2007, Columns: 256


In [3]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

In [60]:
# build model 1: one hidden layer, 12 hidden units fully connected. test accuracy: 0.9268
model1 = Sequential()
model1.add(Dense(64, activation='relu', use_bias=True,kernel_initializer='uniform',
                 bias_initializer='random_uniform',input_shape=(256,)))
model1.add(Dropout(0.1))
model1.add(Dense(n_classes, activation='softmax'))
model1.summary()

model1.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])

history = model1.fit(x_train, y_train,
                    epochs=30,
                    verbose=1,
                    batch_size=300,
                    validation_split=0.1)
score = model1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_93 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_39 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_94 (Dense)             (None, 10)                650       
Total params: 17,098
Trainable params: 17,098
Non-trainable params: 0
_________________________________________________________________
Train on 6561 samples, validate on 730 samples
Epoch 1/30
6561/6561 [==============================] - 6s 989us/step - loss: 1.0493 - acc: 0.7110 - val_loss: 0.4758 - val_acc: 0.8918
Epoch 2/30
6561/6561 [==============================] - 0s 42us/step - loss: 0.4292 - acc: 0.8898 - val_loss: 0.3038 - val_acc: 0.9219
Epoch 3/30
6561/6561 [==============================] - 0s 42us/step - loss: 0.3083 

In [19]:
import numpy as np
from keras.layers import Conv1D, MaxPooling1D,Embedding, Activation

X_train = np.expand_dims(x_train, axis=2)
X_test = np.expand_dims(x_test,axis=2)


In [7]:
# build model 2: two hidden layers locally connected. test accuracy: 0.9272

model2 = Sequential()
model2.add(Conv1D(64,9,activation = "sigmoid",input_shape=(256,1)))
model2.add(Dropout(0.1))
model2.add(Conv1D(16,25,activation = "sigmoid"))
model2.add(Dropout(0.1))
model2.add(Flatten())
model2.add(Dense(n_classes,activation='softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer="Adadelta",
              metrics=['accuracy'])

history = model2.fit(X_train, y_train,
                    epochs=epochs,
                    verbose=1,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test))
score = model2.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 7291 samples, validate on 2007 samples
Epoch 1/12
7291/7291 [==============================] - 31s 4ms/step - loss: 2.2484 - acc: 0.2020 - val_loss: 2.0141 - val_acc: 0.2825
Epoch 2/12
7291/7291 [==============================] - 31s 4ms/step - loss: 1.0507 - acc: 0.6701 - val_loss: 0.5525 - val_acc: 0.8286
Epoch 3/12
7291/7291 [==============================] - 30s 4ms/step - loss: 0.3372 - acc: 0.8995 - val_loss: 0.4320 - val_acc: 0.8724
Epoch 4/12
7291/7291 [==============================] - 30s 4ms/step - loss: 0.2505 - acc: 0.9276 - val_loss: 0.3667 - val_acc: 0.9013
Epoch 5/12
7291/7291 [==============================] - 30s 4ms/step - loss: 0.2119 - acc: 0.9407 - val_loss: 0.3393 - val_acc: 0.9138
Epoch 6/12
7291/7291 [==============================] - 31s 4ms/step - loss: 0.1867 - acc: 0.9490 - val_loss: 0.3220 - val_acc: 0.9193
Epoch 7/12
7291/7291 [==============================] - 31s 4ms/step - loss: 0.1637 - acc: 0.9525 - val_loss: 0.3200 - val_acc: 0.9148
Epoch 8

In [26]:
# build model 3: two hidden layers, locally connected with weight sharing

import numpy as np
from keras.layers import Conv1D, Input, Activation
from keras.models import Model

main_input = Input(shape=(256,1), dtype='float32', name='main_input')

x1 = Conv1D(64,9,activation = "sigmoid",use_bias = True,
            bias_initializer = "random_uniform")(main_input)
x2 = Conv1D(64,9,activation = "sigmoid",use_bias = True,
            bias_initializer = "random_uniform")(main_input)

x3 = LocallyConnected1D(16, 25,activation = "sigmoid",use_bias = True)(x1)
x4 = LocallyConnected1D(16, 25,activation = "sigmoid",use_bias = True)(x2)

x = keras.layers.concatenate([x3, x4])
x_flatten = Flatten()(x)
main_output = Dense(n_classes, activation='softmax',name='main_output')(x_flatten)

model3 = Model(inputs=main_input, outputs=main_output)

model3.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])

history = model3.fit(X_train, y_train,
                    epochs=epochs,
                    verbose=1,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test))
score = model3.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 7291 samples, validate on 2007 samples
Epoch 1/12
7291/7291 [==============================] - 711s 97ms/step - loss: 13.3139 - acc: 0.0911 - val_loss: 13.8143 - val_acc: 0.0732
Epoch 2/12
7291/7291 [==============================] - 623s 85ms/step - loss: 13.2521 - acc: 0.0905 - val_loss: 12.4872 - val_acc: 0.0732
Epoch 3/12
7291/7291 [==============================] - 814s 112ms/step - loss: 12.3234 - acc: 0.1040 - val_loss: 12.4428 - val_acc: 0.0827
Epoch 4/12
7291/7291 [==============================] - 749s 103ms/step - loss: 12.1256 - acc: 0.1613 - val_loss: 12.4312 - val_acc: 0.1096
Epoch 5/12
7291/7291 [==============================] - 651s 89ms/step - loss: 12.0381 - acc: 0.1954 - val_loss: 12.2554 - val_acc: 0.2167
Epoch 6/12
7291/7291 [==============================] - 653s 90ms/step - loss: 11.9600 - acc: 0.2215 - val_loss: 12.2454 - val_acc: 0.2277
Epoch 7/12
7291/7291 [==============================] - 649s 89ms/step - loss: 11.9283 - acc: 0.2369 - val_loss: 12.

In [91]:
#Test accuracy: 0.9427
from keras.optimizers import SGD
model4 = Sequential()
model4.add(Conv1D(64,9,activation = "relu",use_bias = True,bias_initializer='random_uniform',
                 kernel_initializer='uniform',input_shape=(256,1)))
model4.add(Flatten())
model4.add(Dense(100, activation='relu', use_bias=True,bias_initializer='random_uniform',
                 kernel_initializer='uniform'))
model4.add(Dense(100, activation='tanh', use_bias=True,bias_initializer='random_uniform',
                 kernel_initializer='uniform'))
model4.add(Dropout(0.2))
model4.add(Dense(n_classes, activation='softmax'))
#sgd = SGD(lr = 0.001, decay = 1e-7, momentum = 0.9)


model4.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])

history = model4.fit(X_train, y_train,
                    epochs=30,
                    verbose=1,
                    batch_size=300,
                    validation_split=0.1)

score = model4.evaluate(X_test,y_test,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 6561 samples, validate on 730 samples
Epoch 1/30
6561/6561 [==============================] - 19s 3ms/step - loss: 0.9945 - acc: 0.6758 - val_loss: 0.3721 - val_acc: 0.8945
Epoch 2/30
6561/6561 [==============================] - 5s 716us/step - loss: 0.3263 - acc: 0.9023 - val_loss: 0.2559 - val_acc: 0.9178
Epoch 3/30
6561/6561 [==============================] - 5s 737us/step - loss: 0.1974 - acc: 0.9425 - val_loss: 0.1896 - val_acc: 0.9356
Epoch 4/30
6561/6561 [==============================] - 5s 791us/step - loss: 0.1687 - acc: 0.9505 - val_loss: 0.1636 - val_acc: 0.9548
Epoch 5/30
6561/6561 [==============================] - 5s 781us/step - loss: 0.1146 - acc: 0.9665 - val_loss: 0.1568 - val_acc: 0.9507
Epoch 6/30
6561/6561 [==============================] - 5s 719us/step - loss: 0.0917 - acc: 0.9730 - val_loss: 0.1398 - val_acc: 0.9575
Epoch 7/30
6561/6561 [==============================] - 5s 826us/step - loss: 0.0668 - acc: 0.9809 - val_loss: 0.1614 - val_acc: 0.9521
Ep

In [94]:
# test accuracy: 0.9347
from keras.optimizers import SGD
model4 = Sequential()
model4.add(Dense(100,activation = "relu",use_bias = True,bias_initializer='random_uniform',
                 kernel_initializer='uniform',input_shape=(256,)))
#model4.add(Dropout(0.2))
model4.add(Dense(100, activation='relu', use_bias=True,bias_initializer='random_uniform',
                 kernel_initializer='uniform'))
#model4.add(Dropout(0.2))
model4.add(Dense(100, activation='tanh', use_bias=True,bias_initializer='random_uniform',
                 kernel_initializer='uniform'))
model4.add(Dropout(0.2))
model4.add(Dense(n_classes, activation='softmax'))
#sgd = SGD(lr = 0.001, decay = 1e-7, momentum = 0.9)


model4.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])

history = model4.fit(x_train, y_train,
                    epochs=30,
                    verbose=1,
                    batch_size=300,
                    validation_split=0.1)

score = model4.evaluate(x_test,y_test,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 6561 samples, validate on 730 samples
Epoch 1/30
6561/6561 [==============================] - 13s 2ms/step - loss: 1.2342 - acc: 0.6170 - val_loss: 0.6646 - val_acc: 0.7959
Epoch 2/30
6561/6561 [==============================] - 1s 105us/step - loss: 0.4577 - acc: 0.8679 - val_loss: 0.3352 - val_acc: 0.8932
Epoch 3/30
6561/6561 [==============================] - 1s 98us/step - loss: 0.2981 - acc: 0.9142 - val_loss: 0.2746 - val_acc: 0.9096
Epoch 4/30
6561/6561 [==============================] - 0s 73us/step - loss: 0.2406 - acc: 0.9317 - val_loss: 0.2179 - val_acc: 0.9342
Epoch 5/30
6561/6561 [==============================] - 1s 77us/step - loss: 0.2006 - acc: 0.9415 - val_loss: 0.2279 - val_acc: 0.9301
Epoch 6/30
6561/6561 [==============================] - 0s 70us/step - loss: 0.1818 - acc: 0.9489 - val_loss: 0.1955 - val_acc: 0.9438
Epoch 7/30
6561/6561 [==============================] - 0s 76us/step - loss: 0.1557 - acc: 0.9552 - val_loss: 0.1897 - val_acc: 0.9411
Epoch 8

In [98]:
# Test accuracy: 0.9527
model5 = Sequential()
model5.add(Conv1D(64,9,activation = "relu",use_bias = True,bias_initializer='random_uniform',
                 kernel_initializer='uniform',input_shape=(256,1)))

model5.add(Conv1D(16,25, activation='relu', use_bias=True,bias_initializer='random_uniform',
                 kernel_initializer='uniform'))
model5.add(Dropout(0.1))
model5.add(Flatten())
model5.add(Dense(100, activation='relu', use_bias=True,bias_initializer='random_uniform',
                 kernel_initializer='uniform'))
model5.add(Dense(100, activation='tanh', use_bias=True,bias_initializer='random_uniform',
                 kernel_initializer='uniform'))
model5.add(Dropout(0.1))
model5.add(Dense(n_classes, activation='softmax'))



model5.compile(loss='categorical_crossentropy',
              optimizer="rmsprop",
              metrics=['accuracy'])

history = model5.fit(X_train, y_train,
                    epochs=30,
                    verbose=1,
                    batch_size=300,
                    validation_split=0.1)

score = model5.evaluate(X_test,y_test,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 6561 samples, validate on 730 samples
Epoch 1/30
6561/6561 [==============================] - 30s 5ms/step - loss: 1.0817 - acc: 0.6525 - val_loss: 0.3603 - val_acc: 0.8918
Epoch 2/30
6561/6561 [==============================] - 18s 3ms/step - loss: 0.3134 - acc: 0.9040 - val_loss: 0.2706 - val_acc: 0.9178
Epoch 3/30
6561/6561 [==============================] - 18s 3ms/step - loss: 0.2208 - acc: 0.9357 - val_loss: 0.2315 - val_acc: 0.9315
Epoch 4/30
6561/6561 [==============================] - 19s 3ms/step - loss: 0.1634 - acc: 0.9556 - val_loss: 0.1391 - val_acc: 0.9548
Epoch 5/30
6561/6561 [==============================] - 19s 3ms/step - loss: 0.1269 - acc: 0.9625 - val_loss: 0.1340 - val_acc: 0.9616
Epoch 6/30
6561/6561 [==============================] - 18s 3ms/step - loss: 0.1027 - acc: 0.9677 - val_loss: 0.1106 - val_acc: 0.9726
Epoch 7/30
6561/6561 [==============================] - 18s 3ms/step - loss: 0.0808 - acc: 0.9758 - val_loss: 0.0867 - val_acc: 0.9685
Epoch 8/